## Семинар 15: "Обучение с подкреплением 2"

ФИО: Устинов Иван Вадимович

###  FrozenLake


<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>



In [1]:
import gym
import numpy as np 
import random

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [2]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

## Задание 1.
Подберите значения alpha и epsilon и найдите приближение оптимальной Q-функции для Frozen Lake.

In [3]:

class QLearn:
    def __init__(self, actions, epsilon=0.1, alpha=0.2, gamma=0.9):
        self.q = {}
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.actions = actions

    def getQ(self, state, action):
        return self.q.get((state, action), 0.0)

    def learnQ(self, state, action, reward, value):
        oldv = self.q.get((state, action), None)

        if oldv is None:
            self.q[(state, action)] = reward
        else:
            self.q[(state, action)] = oldv + self.alpha * (value - oldv)

    def chooseAction(self, state):
        if random.random() < self.epsilon:
            action = random.choice(self.actions)
        else:
            q = [self.getQ(state, a) for a in self.actions]
            maxQ = max(q)
            count = q.count(maxQ)
            if count > 1:
                best = [i for i in range(len(self.actions)) if q[i] == maxQ]
                i = random.choice(best)
            else:
                i = q.index(maxQ)

            action = self.actions[i]
        return action

    def learn(self, state1, action1, reward, state2):
        maxqnew = max([self.getQ(state2, a) for a in self.actions])
        self.learnQ(state1, action1, reward, reward + self.gamma*maxqnew)

In [4]:
qlearn = QLearn(actions=range(env.env.nA), gamma=1, epsilon=0.1, alpha=0.2)

In [4]:
def run_episode_qlearn_learn(env, qlearn, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        action = qlearn.chooseAction(obs)
        obs_new, reward, done, _ = env.step(action)
        qlearn.learn(obs, action, reward, obs_new)
        obs = obs_new
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [5]:
def run_episode_qlearn(env, qlearn, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        action = qlearn.chooseAction(obs)
        obs_new, reward, done, _ = env.step(action)
        obs = obs_new
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [6]:
def evaluate_qlearn(env, qlearn, gamma = 1.0,  n = 100):
    scores = [
            run_episode_qlearn(env, qlearn, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [8]:
env = gym.make("FrozenLake-v0")

In [39]:
gamma = 0.98
qlearn = QLearn(actions=range(env.env.nA), gamma=gamma, epsilon=0.02, alpha=0.3)

for i in range(10000):
    run_episode_qlearn_learn(env, qlearn, gamma=gamma)
    
evaluate_qlearn(env, qlearn, gamma=gamma,  n=1000)

0.35142174645041141

In [28]:
gamma = 1
qlearn = QLearn(actions=range(env.env.nA), gamma=gamma, epsilon=0.01, alpha=0.2)

for i in range(10000):
    run_episode_qlearn_learn(env, qlearn, gamma=gamma)
    
evaluate_qlearn(env, qlearn, gamma=gamma,  n=1000)

0.68500000000000005

## Задание 2.
Обучите сеть DQN для среды http://gym.openai.com/envs/Pong-v0/ 

In [8]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import cv2
import torch.optim as optim
import time

In [11]:
class MemoryReplay(object):

    def __init__(self,
                 max_size=10000,
                 bs=64,
                 im_size=84,
                 stack=4):

        self.s = np.zeros((max_size, stack+1, im_size, im_size), dtype=np.float32)
        self.r = np.zeros(max_size, dtype=np.float32)
        self.a = np.zeros(max_size, dtype=np.int32)
        self.done = np.array([True]*max_size)

        self.max_size = max_size
        self.bs = bs
        self._cursor = None
        self.total_idx = list(range(self.max_size))


    def put(self, sras):

        if self._cursor == (self.max_size-1) or self._cursor is None :
            self._cursor = 0
        else:
            self._cursor += 1

        self.s[self._cursor] = sras[0]
        self.a[self._cursor] = sras[1]
        self.r[self._cursor] = sras[2]
        self.done[self._cursor] = sras[3]


    def batch(self):

        sample_idx = random.sample(self.total_idx, self.bs)
        s = self.s[sample_idx, :4]
        a = self.a[sample_idx]
        r = self.r[sample_idx]
        ss = self.s[sample_idx, 1:]
        done = self.done[sample_idx]

        return s, a, r, ss, done

In [10]:
class DQN(nn.Module):

    def __init__(self):

        super(DQN, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(4, 8, 8, 4, 0),
            nn.ReLU(inplace=True),
            nn.Conv2d(8, 16, 4, 2, 0)
        )
        self.classifier = nn.Sequential(nn.Linear(16, 12),
                                        nn.ReLU(),
                                        nn.Linear(12, 3)
                                       )

    def forward(self, x):
        x = self.model(x).squeeze()
        x = x.view(-1, 16)
        x = self.classifier(x)
        return x

In [12]:
class Pong(object):

    def __init__(self):

        self.env = gym.make('Pong-v0')
        self.current_phi = None
        self.reset()

    def step(self, action):

        obs, r, done, info = self.env.step(action)
        obs = self._rbg2gray(obs)
        phi_next = self._phi(obs)

        phi_phi = np.vstack([self.current_phi, obs[np.newaxis]])
        self.current_phi = phi_next

        return phi_phi, r, done

    def reset(self):
        x = self.env.reset()
        x = self._rbg2gray(x)
        phi = np.stack([x, x, x, x])
        self.current_phi = phi

        return phi

    def _rbg2gray(self, x):
        x = x.astype('float32')
        x = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
        x = cv2.resize(x, (84, 84))/127.5 - 1.

        return x

    def _phi(self, x):

        new_phi = np.zeros((4, 84, 84), dtype=np.float32)
        new_phi[:3] = self.current_phi[1:]
        new_phi[-1] = x

        return new_phi

    def display(self):
        self.env.render()

In [13]:
def sample_action(env, agent, var_phi, epsilon):

    if random.uniform(0, 1) > epsilon:
        phi = env.current_phi
        var_phi.data.copy_(torch.from_numpy(phi))

        q_values = agent(var_phi)
        max_q, act_index = q_values.max(dim=0)
        act_index = np.asscalar(act_index.data.cpu().numpy())
    else:
        act_index = random.randrange(3)

    return act_index

In [14]:
def run_episode_dqn(pong, dqn, gamma = 1.0, render = False):
    VALID_ACTION = [0, 2, 5]
    obs = env.reset()
    var_phi = Variable(torch.Tensor(1, 4, 84, 84), volatile=True)
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        act_index = sample_action(pong, dqn, var_phi)
        obs_new, reward, done = pong.step(VALID_ACTION[act_index])
        obs = obs_new
        #total_reward += (gamma ** step_idx * reward)
        total_reward += reward
        step_idx += 1
        if done:
            break
    return total_reward

In [15]:
def evaluate_dqn(env, dqn, gamma = 1.0,  n = 100):
    scores = [
            run_episode_dqn(env, dqn, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [17]:
VALID_ACTION = [0, 3, 4]
GAMMA = 0.99
epsilon = 0.5
memory_size = 2000
max_epoch = 100
batch_size = 64

var_phi = Variable(torch.Tensor(1, 4, 84, 84), volatile=True)

var_batch_phi = Variable(torch.Tensor(batch_size, 4, 84, 84))
var_batch_a = Variable(torch.LongTensor(batch_size, 1), requires_grad=False)
var_batch_r = Variable(torch.Tensor(batch_size, 1))
var_batch_phi_next = Variable(torch.Tensor(batch_size, 4, 84, 84))
var_batch_r_mask = Variable(torch.Tensor(batch_size, 1), requires_grad=False)

pong = Pong()
MP = MemoryReplay(memory_size, batch_size)
dqn = DQN()
target_dqn = DQN()
optimz = optim.RMSprop(dqn.parameters(), lr=0.0025, alpha=0.9, eps=1e-02, momentum=0.0)

/home/ivan.ustinov/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/ivan.ustinov/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((210, 160, 3), 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/ivan.ustinov/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((210, 160, 3), 255) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [19]:
for i in range(memory_size):
    phi = pong.current_phi
    act_index = random.randrange(3)
    phi_next, r, done = pong.step(VALID_ACTION[act_index])
#     pong.display()
    MP.put((phi_next, act_index, r, done))

    if done:
        pong.reset()

In [20]:

score = 0.
avg_score = -21.0

SCORE = []
QVALUE = []
QVALUE_MEAN = []
QVALUE_STD = []

In [22]:
pong.reset()

for epoch in range(max_epoch): 

    while not done:

        optimz.zero_grad()

        act_index = sample_action(pong, dqn, var_phi, epsilon)

        epsilon = (epsilon - 1e-6) if epsilon > 0.1 else  0.1
        phi_next, r, done = pong.step(VALID_ACTION[act_index])
#         pong.display()
        MP.put((phi_next, act_index, r, done))
        r = np.clip(r, -1, 1)
        score += r

        batch_phi, batch_a, batch_r, batch_phi_next, batch_done = MP.batch()
        var_batch_phi_next.data.copy_(torch.from_numpy(batch_phi_next))
        batch_target_q, _ = target_dqn(var_batch_phi_next).max(dim=1)

        mask_index = np.ones((batch_size, 1))
        mask_index[batch_done] = 0.0
        var_batch_r_mask.data.copy_(torch.from_numpy(mask_index))

        var_batch_r.data.copy_(torch.from_numpy(batch_r.reshape((batch_size, 1))))

        y = var_batch_r + batch_target_q.mul(GAMMA).mul(var_batch_r_mask)
        y = y.detach()

        var_batch_phi.data.copy_(torch.from_numpy(batch_phi))
        batch_q = dqn(var_batch_phi)

        var_batch_a.data.copy_(torch.from_numpy(batch_a).long().view(-1, 1))
        batch_q = batch_q.gather(1, var_batch_a)

        loss = y.sub(batch_q).pow(2).mean()
        loss.backward()
        optimz.step()

        QVALUE.append(batch_q.data.cpu().numpy().mean())

    SCORE.append(score)
    QVALUE_MEAN.append(np.mean(QVALUE))
    QVALUE_STD.append(np.std(QVALUE))
    QVALUE = []

    pong.reset()
    done = False
    avg_score = 0.9*avg_score + 0.1*score
    score = 0.0
    print('Epoch: {0}. Avg.Score:{1:6f}'.format(epoch, avg_score))

Epoch: 0. Avg.Score:-20.900000
Epoch: 1. Avg.Score:-20.810000
Epoch: 2. Avg.Score:-20.829000
Epoch: 3. Avg.Score:-20.846100
Epoch: 4. Avg.Score:-20.861490
Epoch: 5. Avg.Score:-20.875341
Epoch: 6. Avg.Score:-20.687807
Epoch: 7. Avg.Score:-20.719026
Epoch: 8. Avg.Score:-20.747124
Epoch: 9. Avg.Score:-20.772411
Epoch: 10. Avg.Score:-20.695170
Epoch: 11. Avg.Score:-20.725653
Epoch: 12. Avg.Score:-20.753088
Epoch: 13. Avg.Score:-20.777779
Epoch: 14. Avg.Score:-20.700001
Epoch: 15. Avg.Score:-20.730001
Epoch: 16. Avg.Score:-20.757001
Epoch: 17. Avg.Score:-20.681301
Epoch: 18. Avg.Score:-20.613171
Epoch: 19. Avg.Score:-20.551854
Epoch: 20. Avg.Score:-20.596668
Epoch: 21. Avg.Score:-20.637001
Epoch: 22. Avg.Score:-20.673301
Epoch: 23. Avg.Score:-20.605971
Epoch: 24. Avg.Score:-20.645374
Epoch: 25. Avg.Score:-20.680837
Epoch: 26. Avg.Score:-20.612753
Epoch: 27. Avg.Score:-20.651478
Epoch: 28. Avg.Score:-20.686330
Epoch: 29. Avg.Score:-20.717697
Epoch: 30. Avg.Score:-20.745927
Epoch: 31. Avg.Sco